<a href="https://colab.research.google.com/github/tylaar1/PICAR-autopilot/blob/main/regression_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# remember to switch to t4 gpu

# Imports

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# 1) DATA PRE-PROCESSING

a) Load in labels + image file paths

b) combine them into one dataframe

c) EDA - spotted and removed erroneous label (speed = 1.42...)

## `cleaned_df` is the final df with all of this completed

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1a) load in labels + image file paths

In [3]:
labels_file_path = '/content/drive/MyDrive/machine-learning-in-science-ii-2025/training_norm.csv'
#labels_file_path = '/content/drive/MyDrive/0. MSc MLiS/google SPRING SEMESTER/1. PHYS4036 MLiS2/MLiS2 Project/KAGGLEDATAmachine-learning-in-science-ii-2025/training_norm.csv'
labels_df = pd.read_csv(labels_file_path, index_col='image_id')

In [4]:
#image_folder_path = '/content/drive/MyDrive/0. MSc MLiS/google SPRING SEMESTER/1. PHYS4036 MLiS2/MLiS2 Project/KAGGLEDATAmachine-learning-in-science-ii-2025/training_data/training_data'
image_folder_path = '/content/drive/MyDrive/machine-learning-in-science-ii-2025/training_data/training_data'
image_file_paths = [
    os.path.join(image_folder_path, f)
    for f in os.listdir(image_folder_path)
    if f.lower().endswith(('.png', '.jpg', '.jpeg'))
]

image_file_paths.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0])) # sorts the files in the right order (1.png, 2.png, 3.png, ...)

imagefilepaths_df = pd.DataFrame(
    image_file_paths,
    columns=['image_file_paths'],
    index=[int(os.path.splitext(os.path.basename(path))[0]) for path in image_file_paths]
)

imagefilepaths_df.index.name = 'image_id'

Checking labels dataframe

In [5]:
labels_df.head()

,angle,speed
image_id,,
1,0.4375,0.0
2,0.8125,1.0
3,0.4375,1.0
4,0.6250,1.0
5,0.5000,0.0


Checking image file paths dataframe - as you can see the file paths are ordered correctly (1.png, 2.png, 3.png, ...)

In [6]:
imagefilepaths_df.head()

,image_file_paths
image_id,
1,/content/drive/MyDrive/machine-learning-in-sci...
2,/content/drive/MyDrive/machine-learning-in-sci...
3,/content/drive/MyDrive/machine-learning-in-sci...
4,/content/drive/MyDrive/machine-learning-in-sci...
5,/content/drive/MyDrive/machine-learning-in-sci...


### 1b) Combine labels and image file paths into one dataframe

In [7]:
merged_df = pd.merge(labels_df, imagefilepaths_df, on='image_id', how='inner')
merged_df['speed'] = merged_df['speed'].round(6) # to get rid of floating point errors

In [8]:
merged_df.head()

,angle,speed,image_file_paths
image_id,,,
1,0.4375,0.0,/content/drive/MyDrive/machine-learning-in-sci...
2,0.8125,1.0,/content/drive/MyDrive/machine-learning-in-sci...
3,0.4375,1.0,/content/drive/MyDrive/machine-learning-in-sci...
4,0.6250,1.0,/content/drive/MyDrive/machine-learning-in-sci...
5,0.5000,0.0,/content/drive/MyDrive/machine-learning-in-sci...


In [9]:
merged_df.loc[3139:3143]

,angle,speed,image_file_paths
image_id,,,
3139,0.750,1.0,/content/drive/MyDrive/machine-learning-in-sci...
3140,0.875,1.0,/content/drive/MyDrive/machine-learning-in-sci...
3142,0.625,0.0,/content/drive/MyDrive/machine-learning-in-sci...
3143,0.625,1.0,/content/drive/MyDrive/machine-learning-in-sci...


The above cell shows that:

 1) the image files and labels match (see image_id and the number at the end of the file path)

 2) the missing rows in labels_df (image_id: 3141, 3999, 4895, 8285, 10171) have been taken care of

### 1c) EDA

In [10]:
merged_df.value_counts('angle')

,count
angle,
0.7500,2123
0.5000,2046
0.6875,2007
0.6250,1963
0.5625,1609
0.4375,1467
0.8125,1147
0.3750,428
0.8750,301


note: imbalance datset

identifying the row with the erroneous speed value

In [11]:
merged_df[merged_df['speed'] == 1.428571]

,angle,speed,image_file_paths
image_id,,,
3884,0.4375,1.428571,/content/drive/MyDrive/machine-learning-in-sci...


we want to remove this row

In [12]:
cleaned_df = merged_df[merged_df['speed'] != 1.428571]
cleaned_df.loc[3882:3886]

,angle,speed,image_file_paths
image_id,,,
3882,0.5625,1.0,/content/drive/MyDrive/machine-learning-in-sci...
3883,0.3750,0.0,/content/drive/MyDrive/machine-learning-in-sci...
3885,0.0000,1.0,/content/drive/MyDrive/machine-learning-in-sci...
3886,0.7500,1.0,/content/drive/MyDrive/machine-learning-in-sci...


## convert from pandas to tf


In [13]:
def process_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)  # Use decode_png for PNG images
    image = tf.image.resize(image, (128, 128))  # Resize to uniform shape
    image = image / 255.0  # Normalize pixel values to [0,1]
    return image, label

# Convert DataFrame into a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((cleaned_df["image_file_paths"], cleaned_df["angle"]))

dataset = dataset.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)

dataset = dataset.cache()
dataset = dataset.shuffle(len(cleaned_df))
dataset = dataset.batch(32)
dataset = dataset.prefetch(tf.data.AUTOTUNE)


for images, labels in dataset.take(1):
    print(images.shape, labels.shape)

(32, 128, 128, 3) (32,)


In [14]:
dataset_size = tf.data.experimental.cardinality(dataset).numpy()
train_size = int(0.8 * dataset_size)

# Split into training and test sets
train_dataset = dataset.take(train_size)
validation_dataset = dataset.skip(train_size)

#not using test set as kaggle is our test set

print(f"Train size: {train_size}, Test size: {dataset_size - train_size}")

Train size: 344, Test size: 87


In [15]:
for batch in train_dataset.take(1):
    print(batch)

(<tf.Tensor: shape=(32, 128, 128, 3), dtype=float32, numpy=
array([[[[1.31556377e-01, 1.31556377e-01, 8.62132385e-02],
         [1.21875003e-01, 1.30024508e-01, 8.47426504e-02],
         [1.32291660e-01, 1.22426473e-01, 8.30269605e-02],
         ...,
         [4.69975509e-02, 4.82843146e-02, 4.65686284e-02],
         [3.41299027e-02, 3.54166664e-02, 3.95833328e-02],
         [3.52941193e-02, 3.52941193e-02, 4.31372561e-02]],

        [[1.37438729e-01, 1.36703432e-01, 8.96446109e-02],
         [1.30330876e-01, 1.30330876e-01, 8.41911733e-02],
         [1.29411772e-01, 1.33455887e-01, 8.03308859e-02],
         ...,
         [3.13725509e-02, 3.67647074e-02, 3.65196094e-02],
         [3.16176489e-02, 3.29656862e-02, 3.29044126e-02],
         [3.13725509e-02, 3.92156877e-02, 3.52941193e-02]],

        [[1.39705881e-01, 1.25428915e-01, 6.54411763e-02],
         [1.26225486e-01, 1.15870096e-01, 8.62132385e-02],
         [1.15502454e-01, 1.11151963e-01, 8.78063738e-02],
         ...,
         

In [16]:
dropoutrate = dropoutrate
input_shape = input_shape

mbnet =  tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

model = tf.keras.Sequential([
  mbnet,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dropout(dropoutrate),
  tf.keras.layers.Dense(1, activation='linear')
])
model.build()

mbnet.trainable = False

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128 (Functional)    │ (None, 4, 4, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2)                   │           2,562 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,260,546 (8.62 MB)

 Trainable params: 2,562 (10.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [17]:
with tf.GradientTape() as tape:
  tape.reset() #this resets the gradient tape

In [18]:
LR = 0.001 #learning rate
optimizer = tf.optimizers.Adam(LR) #adam optimiser

@tf.function
def train_step( model, X , Y):
    with tf.GradientTape() as tape:
        pred = model( X )
        current_loss = tf.reduce_mean(tf.losses.MeanSquaredError( Y,  pred))
    grads = tape.gradient(current_loss, model.trainable_variables)
    optimizer.apply_gradients( zip( grads , model.trainable_variables) )
    current_MSE = tf.reduce_mean(tf.metrics.MeanSquaredError(Y, pred))
    return(current_loss, current_MSE)

In [ ]:
niter = 20

tloss = []
tacc = []
vloss = []
vacc = []

for it in range(niter):
    for image_batch, label_batch in train_dataset:
      #for image, label in zip(image_batch, label_batch):
        #print(image)
        #print(label)
        #print(image.shape, label.shape)

        loss, acc = train_step(model, image_batch, label_batch) #run training


    if it % 10 == 0: #log training metrics
      tf.print('iter: ',it, ', loss: {:.3f}, acc: {:.3f}'.format(loss, acc))
      tloss.append(loss)
      tacc.append(acc)
'we should probably switch to balanced accuracy as eval method due to unbalanced data'
#commented out validation for now as it prints for each batch not each epoch massively slowwing process
'''
    if it % 50 == 0: #log validation metrics
      for val_image, val_label in validation_dataset:
        val_pred = model(val_image)
        val_int=tf.cast(val_label, dtype=tf.int32)
        val_loss = tf.reduce_mean(tf.losses.categorical_crossentropy(tf.one_hot(val_int,depth=2) , val_pred))
        val_acc = tf.reduce_mean(tf.metrics.categorical_accuracy(tf.one_hot(val_int,depth=2) , val_pred))
        tf.print('iter: ',it, ', validation loss: {:.3f}, validation acc: {:.3f}'.format(val_loss, val_acc))
        vloss.append(val_loss)
        vacc.append(val_acc)
'''

iter:  0 , loss: 0.255, acc: 0.969


In [ ]:
'''
f, axarr = plt.subplots(1,10)

i = 0
for image_batch, label_batch in dataset.take(1):  # Take one batch
    for image in image_batch:  # Iterate through images in the batch
        if i < 10:  # Only display the first 5 images
            print('image shape: ', np.shape(image))
            tf.print('label:', label_batch[i])  # Print label for the corresponding image
            axarr[i].imshow(image)
            axarr[i].axis('off')
            i += 1
        else:
            break  # Stop after displaying 5 images
        plt.show()
'''

In [ ]:
model.save_weights('/content/drive/MyDrive/machine-learning-in-science-ii-2025/car_frozen_regression.weights.h5')

In [ ]:
tf.keras.backend.clear_session() #Clear keras session

In [ ]:
dropoutrate = dropoutrate
input_shape = input_shape

mbnet =  tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

model = tf.keras.Sequential([
  mbnet,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dropout(dropoutrate),
  tf.keras.layers.Dense(1, activation='linear')
])
model.build()

mbnet.trainable = False

model.summary() # print the model

In [ ]:
model.load_weights('/content/drive/MyDrive/machine-learning-in-science-ii-2025/car_frozen_regression.weights.h5')

In [ ]:
mbnet.trainable = True
model.summary()

In [ ]:
LR = 0.0001
optimizer = tf.optimizers.Adam(LR) #adam optimiser


In [ ]:
niter = 20

tloss = []
tacc = []
vloss = []
vacc = []

for it in range(niter):
    for image, label in train_dataset:
        loss, acc = train_step(model, image_batch, label_batch)

    if it % 10 == 0: #log training metrics
      tf.print('iter: ',it, ', loss: {:.3f}, acc: {:.3f}'.format(loss, acc))
      tloss.append(loss)
      tacc.append(acc)
'''
    if it % 50 == 0: #log validation metrics
      for val_image, val_label in valid:
        val_pred = model(val_image)
        val_loss = tf.reduce_mean(tf.losses.categorical_crossentropy(tf.one_hot(val_label,depth=2) , val_pred))
        val_acc = tf.reduce_mean(tf.metrics.categorical_accuracy(tf.one_hot(val_label,depth=2) , val_pred))
        tf.print('iter: ',it, ', validation loss: {:.3f}, validation acc: {:.3f}'.format(val_loss, val_acc))
        vloss.append(val_loss)
        vacc.append(val_acc)
'''

In [ ]:
model.save_weights('/content/drive/MyDrive/machine-learning-in-science-ii-2025/car_unfrozen_regression.weights.h5')

In [ ]:
image_folder_path = '/content/drive/MyDrive/machine-learning-in-science-ii-2025/test_data/test_data'
image_file_paths = [
    os.path.join(image_folder_path, f)
    for f in os.listdir(image_folder_path)
    if f.lower().endswith(('.png', '.jpg', '.jpeg'))
]

image_file_paths.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0])) # sorts the files in the right order (1.png, 2.png, 3.png, ...)

imagefilepaths_df = pd.DataFrame(
    image_file_paths,
    columns=['image_file_paths'],
    index=[int(os.path.splitext(os.path.basename(path))[0]) for path in image_file_paths]
)

imagefilepaths_df.index.name = 'image_id'
imagefilepaths_df.head()

In [ ]:
def process_image_no_label(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)  # Use decode_png for PNG images
    image = tf.image.resize(image, (128, 128))  # Resize to uniform shape
    image = image / 255.0  # Normalize pixel values to [0,1]
    return image
test_dataset = tf.data.Dataset.from_tensor_slices((imagefilepaths_df["image_file_paths"]))

test_dataset = test_dataset.map(process_image_no_label, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)


In [ ]:
predictions = model.predict(test_dataset)

In [ ]:
predictions_df.head()

In [ ]:
predictions_df[[0,1]] > 0.5

In [ ]:
predictions_df = pd.DataFrame(predictions)

In [ ]:
predictions_df[predictions_df[[0,1]] > 0.5] = 1
predictions_df[predictions_df[[0,1]] < 0.5] = 0

In [ ]:
predictions_df[0]

In [ ]:
predictions_df[0].value_counts()

# add data augmentation steps here

# transfer learning here